In [ ]:
%pip install wandb
%pip install tensorflow

import tensorflow as tf
import pprint

# Log in to your W&B account
import wandb
wandb.login()
# API: **PRIVATE_API**

# Loading the MNIST dataset in one line
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize and reshape
x_train_normalized = x_train/255
x_test_normalized = x_test/255
x_train= x_train_normalized.reshape(-1, 28, 28, 1)
x_test = x_test_normalized.reshape(-1, 28, 28, 1)

# Printing the shape
print('x_train:', x_train.shape)
print('y_train:', y_train.shape)
print('x_test:', x_test.shape)
print('y_test:', y_test.shape)

In [ ]:
# Function to create net
def Net(filter1=28, filter2=28, neuron3=56):
    return tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(28,28,1)),
        tf.keras.layers.Conv2D(filters=filter1, kernel_size=5, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Conv2D(filters=filter2, kernel_size=5, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(neuron3, kernel_regularizer = tf.keras.regularizers.l2(0.07), activation = 'relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.GaussianNoise(0.1),

        tf.keras.layers.Dense(10, activation='softmax')
        ])

model = Net()
model.summary()

def get_optimizer(lr=1e-3, optimizer="adam"):
    "Select optmizer between adam and sgd with momentum"
    if optimizer.lower() == "adam":
        return tf.keras.optimizers.Adam(learning_rate=lr)
    if optimizer.lower() == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.1)

def train(model, batch_size=64, epochs=10, lr=1e-3, optimizer='adam', log_freq='epoch'):

    # Compile modell
    tf.keras.backend.clear_session()
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
    sampleID = 100
    loss_fn(y_train[:1], model(x_train[sampleID-1:sampleID]).numpy()).numpy()

    model.compile(loss=loss_fn,
                  optimizer=get_optimizer(lr, optimizer),
                  metrics=['accuracy'])

    # Callback setup
    wandb_callbacks = [wandb.keras.WandbCallback(log_freq=log_freq,save_model=(False))]

    # Train your model
    model.fit(
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data = (x_test, y_test),
        callbacks = [wandb_callbacks]
        )

# Configure Sweep
sweep_config = {
    'method': 'bayes',
    'metric': {
        'goal': 'maximize',
        'name': 'val_accuracy'
    }
}
parameters_dict = {
    'optimizer': {
        'value': 'adam'
        },
    'epochs': {
        'value': 50
        },
    'batch_size' : {
        'values': [64, 128, 256, 512, 1024, 2048]
        },
    'learning_rate': {
        'values': [0.5e-3, 1e-3, 2e-3, 3e-3, 4e-3, 5e-3, 6e-3]
    },
     'filter1': {
        'values': list(range(4,36,2))
    },
    'filter2': {
        'values': list(range(14,36,2))
    },
    'neuron3': {
        'values': list(range(12,256,8))
    }
}

sweep_config['parameters'] = parameters_dict
pprint.pprint(sweep_config)

def sweep_train(config_defaults=None):

    with wandb.init(config=config_defaults):

        # specify the configuration
        wandb.config.architecture_name = "V01"
        wandb.config.dataset_name = "MNIST"

        # initialize model
        model = Net(wandb.config.neurons1, wandb.config.neurons2, wandb.config.neurons3)

        # start training
        train(model,
              wandb.config.batch_size,
              wandb.config.epochs,
              wandb.config.learning_rate,
              wandb.config.optimizer)

sweep_id = wandb.sweep(sweep_config, project="filter_and_neuron")
wandb.agent(sweep_id, function=sweep_train, count=100) # sweep 100 configuration